In [1]:
#preprocesamiento datos de aprehendidos/detenidos
import pandas as pd
import numpy as np

#CARGA DE DATOS

nombre_archivo = 'detenidosaprehendidos/dataset/mdi_detenidosaprehendidos_pm_2025_enero_octubre.xlsx'

try:
    print("Cargando archivo Excel... esto puede tardar unos segundos...")
    df_apre = pd.read_excel(
        nombre_archivo,
        sheet_name=1,
        dtype={'codigo_parroquia': str},
        engine='openpyxl'
    )
    print(f"Registros totales: {len(df_apre)}")

except Exception as e:
    print("Error cargando el Excel:", e)



Cargando archivo Excel... esto puede tardar unos segundos...
Registros totales: 61149


In [ ]:
#GRID ESPACIAL, PARA TENER UN RANGO DE COORDENADAS
df_apre_clean["lat_grid"] = df_apre_clean["latitud"].round(3)
df_apre_clean["lon_grid"] = df_apre_clean["longitud"].round(3)

#Conteo de delito por grid, target
grouped = (
    df_apre_clean
    .groupby(["lat_grid", "lon_grid", "fecha"])
    .size()
    .reset_index(name="conteo_delitos")
)

df_apre_clean = df_apre_clean.merge(
    grouped,
    on=["lat_grid", "lon_grid", "fecha"],
    how="left"
)